# Driver Monitoring System Using Machine Learning


To develop a driver monitoring system with the objective of building a Machine learning algorithm for tracking of drivers behavioral attributes.

## Dataset Used
 
https://ibug.doc.ic.ac.uk/resources/300-W/

• The 300-W is a face dataset that consists of 300 Indoor and 300 Outdoor in-the-wild
images. Itcovers a large variation of identity, expression, illumination conditions, pose,
occlusion and facesize.

• The images were downloaded from google.com by making queries such as party,conference,
protests, football and celebrities.


## Methodolgy:

### Measuring EAR
Eye Aspect Ratio (EAR) is used to detect a blink which is the the ratio of vertical distance
of the lower and upper eyelids to the horizontal length of an eye. During the eye blinks, the
vertical distance of lower and upper eyelids decreases similarly in opened eye after a blink
the distance between the respective eyelids tends to increases.

<p><img style=" max-width:800px ;max-height:1200px" src="images/EAR.png"></p>




In [8]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2
import os
import winsound

In [9]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [10]:
thresh = 0.25
frame_check = 20
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

In [11]:
#from google.colab import drive
#drive.mount('/content/drive')

In [12]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [13]:
#cap=cv2.VideoCapture("http://192.168.29.9:4747/video")


cap=cv2.VideoCapture(0)

flag=0
while True:
    ret, frame=cap.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        if ear < thresh:
            flag += 1
            #print (flag)
            if flag >= frame_check:
                cv2.putText(frame, "****************ALERT!****************", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "****************ALERT!****************", (10,325),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                winsound.Beep(800, 250)
                
               
        else:
            flag = 0
    cv2.imshow("Camera", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
    break
